In [ ]:
import sys
from pathlib import Path

if '__file__' in globals():
    # Navigate up 4 levels from file location to repo root
    repo_root = Path(__file__).resolve().parent.parent.parent.parent
else:
    # Test two execution scenarios
    cwd = Path.cwd()
    if (cwd / 'src').exists():
        # Executing from repo root (right above src)
        repo_root = cwd
    else:
        # Executing from notebooks directory, go up 3 levels
        repo_root = cwd.parent.parent.parent

repo_root_path = str(repo_root.resolve())
if repo_root_path not in sys.path:
    sys.path.insert(0, repo_root_path)
    print(f'{repo_root_path} added to sys path')

In [ ]:
from src.etl.common.config import *
from src.etl.common.table_updater import *

In [ ]:
session = get_session()

In [ ]:
table_name = 'dim_employee_profile'

In [ ]:
batch_id = f"{table_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{str(uuid.uuid4())[:8]}"
print(f'Batch {batch_id}')

In [ ]:
session.sql(f"""
CREATE OR REPLACE VIEW learning_db.unit_test.vw_dim_employee_profile AS
SELECT
    employee_id,
    first_name,
    last_name,
    department,
    SHA1(CONCAT_WS('|',
        COALESCE(CAST(first_name as STRING), '|'),
        COALESCE(CAST(last_name as STRING), '|'),
        COALESCE(CAST(department as STRING), '|')
    )) AS etl_row_hash_value --all but natural composite key
FROM learning_db.unit_test.source_employee_profile
""").collect()

In [ ]:
table_updater = TableUpdater(
    session, 
    table_name, 
    batch_id,
    schema_name='unit_test',
    etl_schema_name='unit_test',
    src_schema_name='unit_test'
)

In [ ]:

table_updater.identify_upserts()

In [ ]:

table_updater.process_table_updates()

In [ ]:

table_updater.process_table_inserts()